In [215]:
import torch
from torch.nn import Conv1d, MaxPool1d, GRU, Linear, CTCLoss
import torch.nn.functional as F
import torch.nn as nn
from tqdm.notebook import tqdm

In [222]:
class ConvolutionalEncoder(nn.Module):
    def __init__(self):
        super(ConvolutionalEncoder, self).__init__()
        self.layer_1 = Conv1d(in_channels=1, out_channels=4, stride=2, kernel_size=5)
        self.layer_2 = Conv1d(in_channels=4, out_channels=16, stride=2, kernel_size=5)
        self.layer_3 = Conv1d(in_channels=16, out_channels=64, stride=2, kernel_size=5)
        self.layer_4 = Conv1d(in_channels=64, out_channels=256, stride=2, kernel_size=19)

    def forward(self, x):
        x = F.relu(self.layer_1(x))
        x = F.relu(self.layer_2(x))
        x = F.relu(self.layer_3(x))
        x = F.relu(self.layer_4(x))
        x = x.permute(0, 2, 1)
        return x
        

In [223]:
class RNNLayers(nn.Module):
    def __init__(self, hidden_size, num_layers, n_classes):
        super(RNNLayers, self).__init__()

        self.bigru = GRU(input_size=hidden_size,
                            hidden_size=hidden_size, num_layers=num_layers, batch_first=True, bidirectional=True)

        self.output = Linear(hidden_size*2, n_classes)

        self.num_layers = num_layers
        self.hidden_size = hidden_size

    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size, device=x.device)
        x, _ = self.bigru(x, h0)

        return self.output(x)


In [229]:
class Model(nn.Module):
    """Batch size 1 fixed"""
    def __init__(self, hidden_size=256, n_layers=3, output_classes=20):
        super(Model, self).__init__()
        self.cnn_layers = ConvolutionalEncoder()
        self.rnn_layers = RNNLayers(hidden_size=256, num_layers=n_layers, n_classes=output_classes)

    def forward(self, x):
        x = self.cnn_layers.forward(x)
        x = self.rnn_layers.forward(x)
        return x[0]

In [186]:
conv_encoder = ConvolutionalEncoder()
rnn_layers = RNNLayers(hidden_size=256, num_layers=3, n_classes=10)
ctc = CTCLoss(blank=0, zero_infinity=True)

In [203]:
batch_size = 1
n_channels = 1
n_samples = 30000
output_classes = 20

input_arr = torch.randn([batch_size, n_channels, n_samples])

x = conv_encoder.forward(input_arr)
x = rnn_layers.forward(x)

In [193]:
x.shape

torch.Size([1, 1865, 10])

In [195]:
labels = torch.randint(low=1, high=output_classes, size=(300,))
output_classes = 20

In [ ]:
model_output = x[0]
print(model_output.shape)

# Do we need to softmax them?

torch.Size([1865, 10])


In [ ]:
n_timesteps = model_output.shape[0]
input_lengths = torch.tensor(n_timesteps)
label_lengths = torch.tensor(len(labels))

In [199]:
label_lengths

tensor(300)

In [200]:
loss = ctc(model_output, labels, input_lengths, label_lengths)

In [201]:
loss

tensor(-4.3475, grad_fn=<MeanBackward0>)

In [147]:

# Target are to be un-padded
T = 50      # Input sequence length
C = 20      # Number of classes (including blank)
N = 16      # Batch size
# Initialize random batch of input vectors, for *size = (T,N,C)
input = torch.randn(T, N, C).log_softmax(2).detach().requires_grad_()
input_lengths = torch.full(size=(N,), fill_value=T, dtype=torch.long)

# Initialize random batch of targets (0 = blank, 1:C = classes)
target_lengths = torch.randint(low=1, high=T, size=(N,), dtype=torch.long)
print(target_lengths.shape)
target = torch.randint(low=1, high=C, size=(sum(target_lengths),), dtype=torch.long)
ctc_loss = CTCLoss()
loss = ctc_loss(input, target, input_lengths, target_lengths)

torch.Size([16])


In [148]:
print(input_lengths.shape)

torch.Size([16])


In [149]:
target_lengths = torch.randint(low=1, high=T, size=(), dtype=torch.long)

print(target_lengths)

tensor(32)


In [204]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)
print(f"Running on {device}")

Running on cuda


In [232]:
model = Model()
model.to(device)

Model(
  (cnn_layers): ConvolutionalEncoder(
    (layer_1): Conv1d(1, 4, kernel_size=(5,), stride=(2,))
    (layer_2): Conv1d(4, 16, kernel_size=(5,), stride=(2,))
    (layer_3): Conv1d(16, 64, kernel_size=(5,), stride=(2,))
    (layer_4): Conv1d(64, 256, kernel_size=(19,), stride=(2,))
  )
  (rnn_layers): RNNLayers(
    (bigru): GRU(256, 256, num_layers=3, batch_first=True, bidirectional=True)
    (output): Linear(in_features=512, out_features=20, bias=True)
  )
)

In [233]:


for i in tqdm(range(20000)):
    input_arr = torch.randn([batch_size, n_channels, n_samples]).to(device)
    labels = torch.randint(low=1, high=output_classes, size=(300,)).to(device)

    model_output = model(input_arr)
    n_timesteps = model_output.shape[0]
    input_lengths = torch.tensor(n_timesteps)
    label_lengths = torch.tensor(len(labels))

    loss = ctc(model_output, labels, input_lengths, label_lengths)
    
    if i % 100 == 0:
        print(loss)

  0%|          | 0/20000 [00:00<?, ?it/s]

tensor(-4.0242, device='cuda:0', grad_fn=<MeanBackward0>)


KeyboardInterrupt: 

Okay that works, let's do it with an actual dataset

In [259]:
from training_data import load_training_data, data_preproc, create_label_for_training
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import normalize

# Loading the training data
dataset_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\datasets\synthetic\working_datasets\unnormalized\synth_dataset.pkl"
df = pd.read_pickle(dataset_path)
X, y = load_training_data(dataset_path, column_x='Squiggle', column_y='Motifs', sample=True)
y = create_label_for_training(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)


In [260]:
i=0
for X, y in tqdm(zip(X_train, y_train), total=len(X_train)):
    X = torch.tensor(normalize([X]).flatten(), dtype=torch.float32).view(1, 1, len(X)).to(device)
    y = torch.tensor(y).to(device)

    model_output = model(X)
    n_timesteps = model_output.shape[0]
    input_lengths = torch.tensor(n_timesteps)
    label_lengths = torch.tensor(len(y))

    loss = ctc(model_output, y, input_lengths, label_lengths)
    
    i+=1
    
    if i % 20 == 0:
        print(loss)
    

  0%|          | 0/24 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
j = normalize([seq]).flatten()

In [245]:
import numpy as np

s = np.random.rand(200)

In [246]:
k = torch.tensor(s)

In [249]:
k.view(1, 1, len(k))

tensor([[[0.5469, 0.2545, 0.8719, 0.5503, 0.7073, 0.3194, 0.8805, 0.8818,
          0.8647, 0.3087, 0.7410, 0.0531, 0.1897, 0.4908, 0.1428, 0.3800,
          0.8433, 0.2853, 0.8279, 0.3857, 0.1009, 0.1842, 0.7496, 0.2354,
          0.8592, 0.2822, 0.7880, 0.8870, 0.4443, 0.1175, 0.5885, 0.8236,
          0.7631, 0.3113, 0.2436, 0.5066, 0.1640, 0.6864, 0.8421, 0.3651,
          0.8607, 0.6744, 0.2492, 0.6591, 0.7936, 0.3364, 0.8322, 0.9688,
          0.8056, 0.8590, 0.2440, 0.1395, 0.9453, 0.9588, 0.4300, 0.1000,
          0.1965, 0.1581, 0.0515, 0.7125, 0.9257, 0.0179, 0.5834, 0.5943,
          0.6707, 0.6513, 0.8934, 0.9847, 0.0132, 0.3308, 0.0030, 0.2410,
          0.4152, 0.4932, 0.0391, 0.6750, 0.5782, 0.5437, 0.6792, 0.7210,
          0.8724, 0.1410, 0.1621, 0.2649, 0.6362, 0.9052, 0.3517, 0.7360,
          0.0073, 0.5749, 0.7050, 0.9296, 0.7716, 0.4194, 0.9447, 0.5799,
          0.2465, 0.1199, 0.5421, 0.7371, 0.8201, 0.8907, 0.6800, 0.0643,
          0.2057, 0.2604, 0.0080, 0.33

In [ ]:
s